# Programming Assignment: Social Network Database Fix

## Welcome!

You've reached a practical part of our course, **Generative AI for Software Development - AI-Powered Software and System Design**. This task is all about diving into a database designed for managing a social network. It's a hands-on challenge to test your skills and see how well you can work with AI tools, like a Language Learning Model (LLM).

## Your Mission

Your job is to fix a database code that's not working right. It's supposed to handle a social network with two main parts: a `Person` table and a `Club` table. There's a mistake in the code, and it's up to you to find and correct it.

### Tasks:

1. **Fix the Database:** Hunt down the mistake messing up the database and correct it.
2. **Write Three Functions:** Use the LLM to help you craft three functions. They should:
   - List all members of a specific club.
   - List all friends of a specific person.
   - List all people who consider a specific person their friend.

### Working with the LLM:

- **Ask the LLM for Help:** Use the LLM to guide you through fixing the database and creating your functions.
- **Use Its Advice Wisely:** Remember, the LLM's advice might not always be spot-on. It's up to you to decide what's useful.

## What to Submit:

Please submit this Jupyter notebook containing:
- The corrected database code.
- The three functions you've written.

## How We'll Grade It:

- **Database Accuracy:** We'll check if you've successfully identified and fixed the issue in the database.
- **Functionality of Your Functions:** We'll see if each of your functions is doing exactly what it's supposed to do.

## Tips for Success:

- **Be Clear with Your Questions:** The clearer your questions to the LLM, the better help you'll get.
- **Test the Advice:** Always test out the LLM’s suggestions to ensure they work as expected.
- **Hints**: If you struggle with this assignment, you can check some hints we left to you in the bottom of the assignment!

### Necessary imports

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
import numpy as np
import os

This code is setting up the foundation for interacting with a relational database using SQLAlchemy, a popular Python ORM (Object Relational Mapper). Here’s a breakdown of each part:

### 1. **SQLAlchemy Imports**
```python
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Table
```
- **`create_engine`**: Used to establish a connection to the database. You will pass a database URL (e.g., for SQLite, PostgreSQL, MySQL) to create a connection that SQLAlchemy will use to interact with the database.
- **`Column`**: Defines the columns in a database table. You will use this to specify the attributes of your model (e.g., ID, name) and their types.
- **`Integer`, `String`**: These are data types for the columns (similar to SQL types). `Integer` represents an integer type, and `String` represents text data.
- **`ForeignKey`**: Used to define a relationship between tables (typically linking a column in one table to the primary key of another table).
- **`Table`**: Can be used to define association tables for many-to-many relationships between two other tables.

```python
from sqlalchemy.ext.declarative import declarative_base
```
- **`declarative_base`**: This is a factory function that creates a base class for declarative class definitions. All your table models (Python classes representing tables) will inherit from this base.

```python
from sqlalchemy.orm import relationship, sessionmaker
```
- **`relationship`**: This function is used to define relationships between tables (models) in SQLAlchemy, such as one-to-many or many-to-many relationships.
- **`sessionmaker`**: This function is used to create session factories. A session in SQLAlchemy is where you handle transactions with the database (querying, updating, etc.).

### 2. **Other Imports**
```python
import numpy as np
import os
```
- **`numpy`**: This library is widely used for numerical computations, but in this code, it's not clear how it’s going to be used yet. It might be used for manipulating numerical data that's later stored or queried from the database.
- **`os`**: This module provides functions for interacting with the operating system. It can be used for file paths, environment variables, and other OS-level operations. Again, its purpose here is not clear yet, but it could be for accessing environment variables (like database credentials) or file handling.

### Summary
This code is preparing to work with a database using SQLAlchemy’s ORM capabilities. It's defining tools to create tables (using the `declarative_base` model) and define relationships between them. The final database interactions, such as establishing connections, defining specific models, or running queries, will likely follow later in the code.

You’re also importing `numpy` and `os`, which might be used in data processing and system interaction later in the project.

In [ ]:
import unittests

## The main code

Below is the primary code for the assignment. It contains **two significant flaws** that could affect the results of certain functions you'll develop for this task. Identifying the bugs directly from the code might be challenging for an LLM, **so you might need to implement some of the functions to better understand the issues**. Your analytical skills will be crucial. Note that although you are required to create three functions, your submission will be evaluated on four aspects, one of which is the accuracy of your database. Ensure that you address the issues in the code provided!

**If you need to start over with a clean version of this assignment, there's a folder named `backup_data` where you can find a fresh copy.**

In [ ]:
def load_dataset(path = "./"):
    """Loads the dataset
    
    RUN THE FUNCTION WITHOUT ANY PARAMETER. 
    THE PARAMETER IS FOR GRADING PURPOSES ONLY.
    
    """
    ## DO NOT CHANGE THIS PART ##
    # To ensure reproducibility
    
    np.random.seed(42)
    
    ## DO NOT CHANGE THIS PART ##
    # To avoid populating the same database several times, always exclude it between a new execution.
    if 'social_network.db' in os.listdir(path):
        os.remove('social_network.db')
    
    # You may pass echo=True to output information that can help you
    engine = create_engine(f'sqlite:///{os.path.join(path, "social_network.db")}', echo=False)
    Base = declarative_base()
    
    # Define the friendship association table
    friendships = Table('friendships', Base.metadata,
        Column('person_id', Integer, ForeignKey('people.id'), primary_key=True),
        Column('friend_id', Integer, ForeignKey('people.id'), primary_key=True)
    )
    
    class Person(Base):
        __tablename__ = 'people'
        id = Column(Integer, primary_key=True)
        name = Column(String, primary_key = False)
        age = Column(Integer)
        gender = Column(String)
        location = Column(String)
    
        friends = relationship("Person",
                               secondary=friendships,
                               primaryjoin=(friendships.c.person_id == id),
                               secondaryjoin=(friendships.c.friend_id == id),
                               backref="friend_of")
        clubs = relationship("Club", secondary="club_members")
    
    class Club(Base):
        __tablename__ = 'clubs'
        id = Column(Integer, primary_key=True)
        description = Column(String)
    
        members = relationship("Person", secondary="club_members")
    
    club_members = Table('club_members', Base.metadata,
        Column('person_id', Integer, ForeignKey('people.id')),
        Column('club_id', Integer, ForeignKey('clubs.id'))
    )
    
    # Create the tables in the database
    Base.metadata.create_all(engine)
    
    # Create a session to interact with the database
    Session = sessionmaker(bind=engine)
    session = Session()
    
    # Sample data
    names = np.array([("Alice", "New York", "Non-binary", 30), 
             ("Bob", "Los Angeles", "Male", 18), 
             ("Charlie", "Chicago", "Male", 60), 
             ("David", "Houston", "Male", 59),
             ("Eve", "Phoenix", "Non-binary", 18), 
             ("Frank", "Los Angeles", "Non-binary", 72), 
             ("Grace", "Chicago", "Female", 35), 
             ("Henry", "Houston", "Male", 21), 
             ("Ivy", "New York", "Female", 46), 
             ("Elena", "Phoenix", "Female", 66)])
    club_descriptions = [
        "Book Club", "Hiking Club", "Chess Club", "Photography Club", "Cooking Club",
        "Music Club", "Gaming Club", "Fitness Club", "Art Club", "Travel Club"
    ]
    
   # Populate People table
    people = []
    for i in range(100):
        person = Person(
            name=np.random.choice(names[:,0]),
            age=np.random.choice(names[:,-1]),
            gender=np.random.choice(names[:,2]),
            location=np.random.choice(names[:,1])
        )
        people.append(person)
        session.add(person)
    
    # Populate Friendships
    for _ in range(200):  # Create 200 random friendships
        person1 = np.random.choice(people)
        person2 = np.random.choice(people)
        if person1 != person2 and person2 not in person1.friends:
            person1.friends.append(person2)
    
    # Populate Clubs table
    clubs = []
    for description in club_descriptions:
        club = Club(description=description)
        clubs.append(club)
        session.add(club)
    
    # Populate Club Members table
    for club in clubs:
        num_members = np.random.randint(5, 10)
        members = np.random.choice(people, num_members)
        club.members.extend(members)
    
    # Commit the changes
    session.commit()
    
    # Close the session
    session.close()
    
    print("Database created and populated successfully!")
    return session, Club, Person, friendships

In [ ]:
# Loads the dataset
session, Club, Person, friendships = load_dataset()

The function `load_dataset` is responsible for creating and populating a SQLite database that simulates a social network, with tables for `people`, their `friendships`, and `clubs` they belong to. Here's a step-by-step breakdown of what this function does:

### 1. **Reproducibility & Cleanup**
```python
np.random.seed(42)
```
- This sets a fixed seed for NumPy's random number generator, ensuring that the random operations performed in the function produce the same result every time it's run.

```python
if 'social_network.db' in os.listdir(path):
    os.remove('social_network.db')
```
- If a database file named `social_network.db` already exists in the given path, it deletes it to avoid re-using or duplicating data across multiple runs of the function.

### 2. **Database Setup**
```python
engine = create_engine(f'sqlite:///{os.path.join(path, "social_network.db")}', echo=False)
Base = declarative_base()
```
- **`create_engine`**: Establishes a connection to a SQLite database called `social_network.db` in the specified path.
- **`Base`**: The base class for defining models (database tables) using the SQLAlchemy ORM.

### 3. **Association Tables and Models**
#### a. **Friendship Table**
```python
friendships = Table('friendships', Base.metadata,
    Column('person_id', Integer, ForeignKey('people.id'), primary_key=True),
    Column('friend_id', Integer, ForeignKey('people.id'), primary_key=True)
)
```
- Defines an association table `friendships` that handles many-to-many relationships between `Person` objects. Each record links two people as friends.

#### b. **Person Model**
```python
class Person(Base):
    __tablename__ = 'people'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    age = Column(Integer)
    gender = Column(String)
    location = Column(String)

    friends = relationship("Person",
                           secondary=friendships,
                           primaryjoin=(friendships.c.person_id == id),
                           secondaryjoin=(friendships.c.friend_id == id),
                           backref="friend_of")
    clubs = relationship("Club", secondary="club_members")
```
- Defines the `Person` model, which maps to the `people` table. Each person has attributes like `id`, `name`, `age`, `gender`, and `location`.
- The `friends` attribute establishes a many-to-many relationship with other `Person` objects through the `friendships` association table.
- The `clubs` attribute links a person to multiple `Club` objects via another association table, `club_members`.

#### c. **Club Model**
```python
class Club(Base):
    __tablename__ = 'clubs'
    id = Column(Integer, primary_key=True)
    description = Column(String)

    members = relationship("Person", secondary="club_members")
```
- Defines the `Club` model, representing a social club. The club has an `id` and a `description`.
- The `members` relationship links a club to multiple `Person` objects via the `club_members` association table.

#### d. **Club Members Table**
```python
club_members = Table('club_members', Base.metadata,
    Column('person_id', Integer, ForeignKey('people.id')),
    Column('club_id', Integer, ForeignKey('clubs.id'))
)
```
- Defines another association table `club_members` to represent the many-to-many relationship between `Person` and `Club`.

### 4. **Table Creation**
```python
Base.metadata.create_all(engine)
```
- This creates the `people`, `clubs`, `friendships`, and `club_members` tables in the database based on the models and relationships defined.

### 5. **Session Creation**
```python
Session = sessionmaker(bind=engine)
session = Session()
```
- A session is created to interact with the database. This session is used to add data to the database and perform operations (such as queries).

### 6. **Data Population**
#### a. **People Table**
```python
names = np.array([...])
```
- A sample dataset of names, locations, genders, and ages is defined.

```python
for i in range(100):
    person = Person(...)
    session.add(person)
```
- The loop creates 100 random `Person` objects by sampling from the predefined `names` array. Each person’s `name`, `age`, `gender`, and `location` are chosen randomly from the array, and the person is added to the session for later insertion into the database.

#### b. **Friendships**
```python
for _ in range(200):
    person1 = np.random.choice(people)
    person2 = np.random.choice(people)
    if person1 != person2 and person2 not in person1.friends:
        person1.friends.append(person2)
```
- Random friendships are created between the `Person` objects. A total of 200 friendships are generated. If the two randomly chosen people aren't already friends, they are added to each other’s `friends` list.

#### c. **Clubs**
```python
clubs = []
for description in club_descriptions:
    club = Club(description=description)
    clubs.append(club)
    session.add(club)
```
- The `Club` objects are created based on predefined descriptions (e.g., "Book Club", "Chess Club") and added to the session.

#### d. **Club Members**
```python
for club in clubs:
    num_members = np.random.randint(5, 10)
    members = np.random.choice(people, num_members)
    club.members.extend(members)
```
- Each club is assigned a random number (between 5 and 10) of members, who are randomly chosen from the `people` dataset.

### 7. **Commit and Close**
```python
session.commit()
session.close()
```
- All the changes are committed to the database, and the session is closed.

### 8. **Return Values**
```python
return session, Club, Person, friendships
```
- The function returns the `session`, `Club`, `Person`, and `friendships` objects, allowing further interaction with the database.

### **Summary**
The function sets up a SQLite database representing a social network, where people can be friends and belong to clubs. It defines models for `Person` and `Club`, creates relationships between them, and populates the database with random people, friendships, and clubs.

### Exercise 1

For this exercise, you are tasked with creating a function named `get_club_members_by_description`. This function should accept a description of a club and a session, and return a list of all its members. **Ensure that this function returns a list containing the defined Person objects.** It must input only a **club description**.

In [ ]:
def get_club_members(session,club_description):
    """
    Returns a list of Person objects who are members of a club given the club's description.
    
    Parameters:
    - club_description (str): The description of the club for which members are to be retrieved.
    
    Returns:
    - List[Person]: A list of Person objects who are members of the specified club.
    """
    # Query the Club table to get the club with the given description
    club = session.query(Club).filter(Club.description == club_description).first()
    
    # If the club does not exist, return an empty list
    if club is None:
        return []
    
    # Return the list of members associated with the found club
    return club.members

This function `get_club_members` is designed to retrieve a list of `Person` objects who are members of a club, based on the club's description. Here's a breakdown of what it does:

### 1. **Function Purpose and Parameters**
- **Parameters:**
  - `session`: An active SQLAlchemy session, which is used to interact with the database.
  - `club_description` (type: `str`): The description of the club whose members are to be retrieved (e.g., "Book Club", "Chess Club").

- **Returns**: 
  - A list of `Person` objects who are members of the club with the specified description. If no such club exists, it returns an empty list.

### 2. **Querying the Database for the Club**
```python
club = session.query(Club).filter(Club.description == club_description).first()
```
- **`session.query(Club)`**: This creates a query on the `Club` table.
- **`filter(Club.description == club_description)`**: The query is filtered to only include clubs where the `description` matches the provided `club_description`.
- **`.first()`**: Retrieves the first result from the query. This ensures that only one `Club` object is returned, even if multiple clubs with the same description exist (though in a well-designed schema, descriptions should be unique).

### 3. **Handling the Case Where the Club Doesn't Exist**
```python
if club is None:
    return []
```
- If no club with the given description exists, `club` will be `None`. In this case, the function returns an empty list, signifying that no members are associated with the given description.

### 4. **Returning the List of Club Members**
```python
return club.members
```
- If a club with the given description is found, the function accesses the `members` attribute of the `club` object.
  - **`club.members`**: This is a list of `Person` objects, representing the people who are members of the club. This list was defined earlier in the `Club` model as part of the many-to-many relationship between `Club` and `Person`.
- The function returns this list of members.

### Example Workflow
If you call this function like:
```python
members = get_club_members(session, "Chess Club")
```
- The function will query the `Club` table to find a club with the description "Chess Club".
- If found, it will return a list of people (i.e., `Person` objects) who are members of that club.
- If no such club exists, it will return an empty list.

### Summary
- The function searches for a club in the database by its description.
- If the club is found, it returns the list of members belonging to that club.
- If no club is found, it returns an empty list.

In [ ]:
# Example usage of the get_club_members function

# Assume the session and all models have been correctly set up and populated as per your initial code

# Fetching members of the "Hiking Club"
hiking_club_members = get_club_members(session,"Hiking Club")

# Printing out the names of all members of the Hiking Club
print("Members of the Hiking Club:")
for person in hiking_club_members:
    print(f"- {person.name}, Age: {person.age}, Location: {person.location}")

In [ ]:
unittests.test_get_club_members(load_dataset, get_club_members)

### Exercise 2

In this exercise, you are required to create a function named `get_friends_of_person`. This function should accept the name of a person and a session, return a list of all their friends. **Ensure that this function returns a list containing the defined Person objects.** The input must be only the **name of a person**.

In [ ]:
def get_friends_of_person(session, person_name):
    """
    Returns a list of Person objects who are friends with the specified person.
    
    Parameters:
    - person_name (str): The name of the person for whom to retrieve friends.
    
    Returns:
    - List[Person]: A list of Person objects who are friends with the specified person.
    """
    # Query the Person table to find the person with the given name
    person = session.query(Person).filter(Person.name == person_name).first()
    
    # If the person does not exist, return an empty list
    if person is None:
        return []
    
    # Return the list of friends associated with the found person
    return person.friends

The function `get_friends_of_person` is designed to retrieve a list of `Person` objects who are friends with a specified person by name. Here's a detailed breakdown of how the function works:

### 1. **Function Purpose and Parameters**
- **Parameters:**
  - `session`: An active SQLAlchemy session, used to interact with the database.
  - `person_name` (type: `str`): The name of the person whose friends are to be retrieved.

- **Returns**: 
  - A list of `Person` objects who are friends with the specified person. If no such person is found, it returns an empty list.

### 2. **Querying the Database for the Person**
```python
person = session.query(Person).filter(Person.name == person_name).first()
```
- **`session.query(Person)`**: Creates a query to search the `Person` table in the database.
- **`filter(Person.name == person_name)`**: Filters the query to only include the person whose `name` matches the provided `person_name`.
- **`.first()`**: Retrieves the first result from the query. This ensures that even if multiple people share the same name (which is possible in some datasets), only the first match is returned. It will return `None` if no match is found.

### 3. **Handling the Case Where the Person Doesn't Exist**
```python
if person is None:
    return []
```
- If no person with the given name is found, `person` will be `None`. In this case, the function returns an empty list, indicating that no friends could be found because the person doesn't exist in the database.

### 4. **Returning the List of Friends**
```python
return person.friends
```
- If a person with the given name is found, the function accesses the `friends` attribute of the `person` object.
  - **`person.friends`**: This is a list of `Person` objects who are friends with the queried person. The `friends` attribute was defined earlier in the `Person` model using a many-to-many relationship with other `Person` objects through the `friendships` table.
- The function returns this list of friends.

### Example Workflow
If you call this function like:
```python
friends = get_friends_of_person(session, "Alice")
```
- The function will query the `Person` table to find a person named "Alice".
- If "Alice" is found, it will return a list of `Person` objects who are friends with Alice.
- If no person named "Alice" exists in the database, it will return an empty list.

### Summary
- The function searches the database for a person by their name.
- If the person is found, it returns the list of their friends.
- If no such person exists, it returns an empty list.

This is a simple and efficient way to retrieve a person’s friends from a database using SQLAlchemy's ORM and its many-to-many relationship capabilities.

In [ ]:
# Example usage of the get_friends_of_person function

# Fetching friends of given name
name = "Bob"

alice_friends = get_friends_of_person(session,name)

# Printing out the names of all friends of Alice
print(f"Friends of {name}:")
for friend in alice_friends:
    print(f"- {friend.name}, Age: {friend.age}, Location: {friend.location}")

In [ ]:
unittests.test_get_friends_of_person(load_dataset, get_friends_of_person)

### Exercise 3

In this exercise, you're tasked with crafting a function called `get_persons_who_consider_them_friend`. This function should take two parameters: the name of an individual and a session. It will return a list of people who count this individual as a friend. It's important to remember that in our database, friendship isn't necessarily mutual. For example, Alice might consider Bob a friend, but Bob might not feel the same way about Alice. **Your function must return a list of Person objects for everyone who considers the input name their friend.** The input to this function should strictly be the **name of the person** you're inquiring about.

In [ ]:
def get_persons_who_consider_them_friend(session, person_name):
    """
    Returns a list of Person objects who consider the specified person as their friend,
    in a scenario where friendships are unidirectional.
    
    Parameters:
    - person_name (str): The name of the person to find who is considered as a friend by others.
    
    Returns:
    - List[Person]: A list of Person objects who consider the specified person as their friend.
    """
    # First, find the person by the given name to get their ID
    person = session.query(Person).filter(Person.name == person_name).first()
    
    # If the person does not exist, return an empty list
    if person is None:
        return []
    
    # Query the friendships table for rows where the friend_id matches the person's ID
    # Then, join with the Person table to get the details of the people who consider them a friend
    friends_of = session.query(Person).join(friendships, Person.id == friendships.c.person_id).filter(friendships.c.friend_id == person.id).all()
    
    return friends_of

The function `get_persons_who_consider_them_friend` is designed to retrieve a list of `Person` objects who consider a specified person as their friend, assuming that friendships are unidirectional. This means that if person A considers person B their friend, it doesn’t necessarily mean person B considers person A their friend.

Here’s a breakdown of how the function works:

### 1. **Function Purpose and Parameters**
- **Parameters:**
  - `session`: An active SQLAlchemy session for interacting with the database.
  - `person_name` (type: `str`): The name of the person who is being checked (i.e., you want to find out who considers this person their friend).

- **Returns**: 
  - A list of `Person` objects who consider the specified person their friend.

### 2. **Querying the Database for the Person by Name**
```python
person = session.query(Person).filter(Person.name == person_name).first()
```
- **`session.query(Person)`**: This creates a query on the `Person` table.
- **`filter(Person.name == person_name)`**: Filters the `Person` table for a record where the name matches the provided `person_name`.
- **`.first()`**: Retrieves the first `Person` object from the query result (or `None` if no match is found).

### 3. **Handling the Case Where the Person Doesn't Exist**
```python
if person is None:
    return []
```
- If no person with the given `person_name` exists in the database, the function returns an empty list because no one could consider a non-existent person their friend.

### 4. **Querying the Database for People Who Consider the Specified Person as a Friend**
```python
friends_of = session.query(Person).join(friendships, Person.id == friendships.c.person_id).filter(friendships.c.friend_id == person.id).all()
```
- **`session.query(Person)`**: Starts a query on the `Person` table, as we want to find other people who consider the specified person as their friend.
- **`join(friendships, Person.id == friendships.c.person_id)`**: Joins the `Person` table with the `friendships` association table. This joins every `Person` with the `friendships` table based on the `person_id` column in the `friendships` table, which represents the person who considers another as a friend.
- **`filter(friendships.c.friend_id == person.id)`**: Filters the joined result to only include rows where the `friend_id` in the `friendships` table matches the `id` of the person we are querying. In this case, we’re checking for people whose `friend_id` is equal to the `id` of the person we’re interested in (the one found earlier by `person_name`).
- **`.all()`**: Retrieves all the `Person` objects that match the query.

### 5. **Returning the List of People Who Consider the Specified Person Their Friend**
```python
return friends_of
```
- The function returns the list of `Person` objects who consider the specified person as their friend.

### Example Workflow
If you call the function like:
```python
people_who_consider_alice_friend = get_persons_who_consider_them_friend(session, "Alice")
```
- First, the function will search for a `Person` object named "Alice".
- If "Alice" is found, it will query the `friendships` table to find all rows where Alice's `id` appears as the `friend_id`.
- Then, it will return the list of people who have Alice as their friend.

### Summary
This function retrieves all the people who consider a given person (identified by their name) as their friend, in a unidirectional friendship scenario. The query uses a join between the `Person` table and the `friendships` association table to find all people whose `friend_id` matches the specified person’s ID. If no such person exists in the database, the function returns an empty list.

In [ ]:
# Example usage of the get_persons_who_consider_them_friend function

# Fetching people who consider given name as their friend
name = 'Bob'

name_friend_of = get_persons_who_consider_them_friend(session, name)

# Printing out the names of all people who consider Alice as their friend
print(f"People who consider {name} as their friend:")
for person in name_friend_of:
    print(f"- {person.name}, Age: {person.age}, Location: {person.location}")

In [ ]:
unittests.test_get_persons_who_consider_them_friend(load_dataset, get_persons_who_consider_them_friend)

## Now test your Dataset!

In [ ]:
unittests.test_load_dataset(load_dataset)

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint 1</b></font>
</summary>
<p>
Check how the persons are being inserted into the dataset! Does the random is necessary?
</p>
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint 2</b></font>
</summary>
<p>
You may have to properly handle how friendships are created. Ask an LLM how you can fix that part.
</p>
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint 3</b></font>
</summary>
<p>
Remember that friendships are not bidirectional! Specify it when asking the LLM to make your last function!
</p>
</details>

Congratulations! You have finished the assignment! Keep up!